In [18]:
import pandas as pd
from os.path import join
from typing import List, Dict
from datetime import datetime
import pathlib
from binreader.binread import MySmspec

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns',10)

def read_smspecs(folder: str, data: str) -> Dict[str, MySmspec]:
    '''data = daily or yearly'''
    dir = pathlib.Path(folder)
    # эталонная выборка
    res=dict()
    for file in dir.rglob('result.smspec'): 
        key=file.parts[-2]
        type = int(key[:3], 2)
        smspec = MySmspec(file)
        df = smspec.get_main(['FGPT', 'FGPR' ], ['FIELD'])
        if data=='yearly':
            df = df[(df.index.month==1) & (df.index.day==1)]
        df = pd.DataFrame({
            # 'date' : df.index, 
            'total' : df["FIELD: FGPT(SM3)"], 
             data : df["FIELD: FGPT(SM3)"].shift(-1)-df["FIELD: FGPT(SM3)"] if data=='yearly' else df["FIELD: FGPR(SM3/DAY)"]
        })
        df=df[df[data].notna()]
        if len(df)>0:
            res[key]=df
    return res

In [20]:
# формируем файлы из смспеков
smspecs=read_smspecs(r'tnav_results', 'daily')
with pd.ExcelWriter(path = 'files/smspecs_full.xlsx') as writer:
    for key in smspecs:
        smspecs[key].to_excel(writer, sheet_name=key)
    writer.save()
    
smspecs=read_smspecs(r'tnav_results', 'yearly')
with pd.ExcelWriter(path = 'files/smspecs_yearly.xlsx') as writer:
    for key in smspecs:
        smspecs[key].to_excel(writer, sheet_name=key)
    writer.save()
    

c:\Users\A_Voronina\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
c:\Users\A_Voronina\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [21]:
import os
dir = pathlib.Path('tnav_results')
for file in dir.rglob('*.*'): 
    if file.name not in ['result.SMSPEC', 'result.UNSMRY']:
        os.remove(file)